In [1]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-c21ckw0x
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-c21ckw0x
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4290 sha256=44dbeaf09884a8e8b4543557215ca23723c0b512d1528a30ebe10d8f70347602
  Stored in directory: /tmp/pip-ephem-wheel-cache-cidz7_zc/wheels/bc/4e/e0/2d86bd15f671dbeb32144013f1159dba09757fde36dc51a963
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:
%%writefile Assignment4A.cu
#include <iostream>
#include <cuda.h>
#include <chrono>

using namespace std;
using namespace std::chrono;


__global__ void multiply(int* A, int* B, int* C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < K) {
        int sum = 0;
        for (int i = 0; i < N; i++) {
            sum += A[row * N + i] * B[i * K + col];
        }
        C[row * K + col] = sum;
    }
}

void initialize(int* matrix, int rows, int cols) {
    for (int i = 0; i < rows * cols; i++) {
        cout << "Enter element " << i + 1 << ": ";
        cin >> matrix[i];
    }
}

void print(int* matrix, int rows, int cols) {
    for (int row = 0; row < rows; row++) {
        for (int col = 0; col < cols; col++) {
            cout << matrix[row * cols + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

void sequentialMultiply(int* A, int* B, int* C, int M, int N, int K) {
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < K; j++) {
            int sum = 0;
            for (int k = 0; k < N; k++) {
                sum += A[i * N + k] * B[k * K + j];
            }
            C[i * K + j] = sum;
        }
    }
}

int main() {
    int M, N, K;
    cout << "Enter the number of rows and columns of the first matrix: ";
    cin >> M >> N;
    cout << "Enter the number of columns of the second matrix: ";
    cin >> K;

    int* A, * B, * C;

    int matrixSize = M * K;
    size_t matrixBytes = matrixSize * sizeof(int);

    A = new int[M * N];
    B = new int[N * K];
    C = new int[M * K];

    initialize(A, M, N);
    initialize(B, N, K);

    cout << "Matrix A: \n";
    print(A, M, N);

    cout << "Matrix B: \n";
    print(B, N, K);

    int* X, * Y, * Z;
    cudaMalloc(&X, M * N * sizeof(int));
    cudaMalloc(&Y, N * K * sizeof(int));
    cudaMalloc(&Z, M * K * sizeof(int));

    cudaMemcpy(X, A, M * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, N * K * sizeof(int), cudaMemcpyHostToDevice);

    int THREADS = 16;
    int BLOCKS = (M + THREADS - 1) / THREADS;

    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    // Sequential multiplication
    auto start = high_resolution_clock::now();
    sequentialMultiply(A, B, C, M, N, K);
    auto stop = high_resolution_clock::now();
    auto seq_duration = duration_cast<microseconds>(stop - start);

    cout << "Sequential Multiplication of matrix A and B: \n";
    print(C, M, K);

    // Parallel multiplication
    start = high_resolution_clock::now();
    multiply<<<blocks, threads>>>(X, Y, Z, M, N, K);
    cudaMemcpy(C, Z, M * K * sizeof(int), cudaMemcpyDeviceToHost);
    stop = high_resolution_clock::now();
    auto par_duration = duration_cast<microseconds>(stop - start);

    cout << "Parallel Multiplication of matrix A and B: \n";
    print(C, M, K);

    cout << "Sequential Multiplication Time: " << seq_duration.count() << " microseconds" << endl;
    cout << "Parallel Multiplication Time: " << par_duration.count() << " microseconds" << endl;

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}

Overwriting Assignment4A.cu


In [9]:
!nvcc Assignment4A.cu -o Assignment4A
!./Assignment4A


Assignment4A.cu(61): warning #177-D: variable "matrixBytes" was declared but never referenced
      size_t matrixBytes = matrixSize * sizeof(int);
             ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Enter the number of rows and columns of the first matrix: 3
3
Enter the number of columns of the second matrix: 3
Enter element 1: 1
Enter element 2: 2
Enter element 3: 3
Enter element 4: 1
Enter element 5: 3
Enter element 6: 6
Enter element 7: 2
Enter element 8: 9
Enter element 9: 5
Enter element 1: 1
Enter element 2: 3
Enter element 3: 1
Enter element 4: 4
Enter element 5: 1
Enter element 6: 4
Enter element 7: 9
Enter element 8: 6
Enter element 9: 10
Matrix A: 
1 2 3 
1 3 6 
2 9 5 

Matrix B: 
1 3 1 
4 1 4 
9 6 10 

Sequential Multiplication of matrix A and B: 
36 23 39 
67 42 73 
83 45 88 

Parallel Multiplication of matrix A and B: 
36 23 39 
67 42 73 
83 45 88 

Sequential Multiplication Time: 0 microseconds
Parallel Multiplication Time: 4 mic